# Practice with the Python ATProtoSDK
This ipython notebook will walk you through the basics of working with the
ATProto python sdk. The content here heavily draws on [these examples](https://github.com/MarshalX/atproto/tree/main/examples)

In [3]:
from atproto import Client
from dotenv import load_dotenv
import os
import pprint


load_dotenv(override=True)
USERNAME = os.getenv("USERNAME")
PW = os.getenv("PW")

## Logging into your account

In [4]:
client = Client()
profile = client.login(USERNAME, PW)
pprint.pprint(profile.__dict__)

{'associated': ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'),
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:yzpplgm5kftdgpf2wsnrbgdn/bafkreih3fpryxoepb44fzyr3sfn32fr7fqqka4kle6h4not7jlwtdvzghe@jpeg',
 'banner': None,
 'created_at': '2025-02-13T17:12:12.845Z',
 'description': None,
 'did': 'did:plc:yzpplgm5kftdgpf2wsnrbgdn',
 'display_name': '',
 'followers_count': 0,
 'follows_count': 1,
 'handle': 'trustylabeler.bsky.social',
 'indexed_at': '2025-02-13T17:12:12.845Z',
 'joined_via_starter_pack': None,
 'labels': [],
 'pinned_post': None,
 'posts_count': 0,
 'py_type': 'app.bsky.actor.defs#profileViewDetailed',
 'viewer': ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState')}


## Working with posts

In [19]:
def post_from_url(client: Client, url: str):
    """
    Retrieve a Bluesky post from its URL
    """
    parts = url.split("/")
    rkey = parts[-1]
    handle = parts[-3]
    return client.get_post(rkey, handle)

post = post_from_url(client, "https://bsky.app/profile/labeler-test.bsky.social/post/3lktj7ewxxv2q")
pprint.pprint(post.value.__dict__)

{'created_at': '2025-03-20T20:14:57.103160+00:00',
 'embed': Main(images=[Image(alt='dog', image=BlobRef(mime_type='image/jpeg', size=169278, ref=IpldLink(link='bafkreibahplioamouecglrcqnshcxzdrwawdtwl5h676d2l7k7xbbti3pa'), py_type='blob'), aspect_ratio=None, py_type='app.bsky.embed.images#image')], py_type='app.bsky.embed.images'),
 'entities': None,
 'facets': None,
 'labels': None,
 'langs': ['en'],
 'py_type': 'app.bsky.feed.post',
 'reply': None,
 'tags': None,
 'text': 'check out this dog!'}


In [20]:
post.value.text

'check out this dog!'

In [18]:
# https://github.com/MarshalX/atproto/blob/main/examples/profile_posts.py
prof_feed = client.get_author_feed(actor="weratedogs.com")
for i, feed_view in enumerate(prof_feed.feed[:100]):
    print(f"Post {i}:", feed_view.post.record.text)

post = prof_feed.feed[0].post
likes_resp = client.get_likes(post.uri, post.cid, limit=10)
print("Likes:", [like.actor.handle for like in likes_resp.likes])

post_thread_resp = client.get_post_thread(post.uri)
print([rep.post.record.text for rep in post_thread_resp.thread.replies[:10]])

Post 0: This puppy decided he wanted breakfast in bed. And also bed in breakfast. 12/10
Post 1: This is George. He just had a dentist appointment and is none too pleased about it. Will be giving you a piece of his mind as soon as the air stops changing colors. 12/10 #SeniorPupSaturday
Post 2: You all funded Fiyero's care in just under 2 hours ensuring he has the best chance at a diagnosis and treatment to live a comfortable life. We’ve removed his post due to how distressing his seizures were. Follow @15outof10.org for pupdates on him and the rest of our fundraiser dogs ❤️‍🩹
Post 3: thank you so much for your help ❤️
Post 4: what Dasia said 🗣️
Post 5: Babe wake up the dogs are still so very good
Post 6: Get 60% off your dog's first order of The Farmers Dog through the link below 🧡 #ad

bit.ly/3p3SABm
Post 7: Here are the Top 5 Dogs of the week!
Post 8: This is Dill. He was found with a broken leg. With your support of @weratedogs.com merch, we sponsored his surgery and he wound up in a

## Followers/following

How might you use this information to investigate/mitigate a harm?

In [7]:

follower_resp = client.get_followers("weratedogs.com", limit=10)
following_resp = client.get_follows("weratedogs.com", limit=10)
print("Followers:", [follower.handle for follower in follower_resp.followers])
print("Following:", [follow.handle for follow in following_resp.follows])



Followers: ['kelly3010.bsky.social', 'homaksu.bsky.social', 'jennifersamule.bsky.social', 'walkerbn.bsky.social', 'cuneyterdem0.bsky.social', 'shinddha.bsky.social', 'yooperann.bsky.social', 'tor37.bsky.social', 'windowtothesoull.bsky.social', 'twocakesup.bsky.social']
Following: ['15outof10.org']


## Exercise: Compute average dog ratings
The WeRateDogs account includes ratings out of 10 within some of its posts.
Write a script that computes the average rating (out of 10) for the 100 most
recent posts from this account. (note that not every post will have a rating)

In [9]:
import re

In [10]:
def extract_score(text):
    # This pattern looks for:
    # \d+ - one or more digits
    # \s* - optional whitespace
    # / - literal forward slash
    # \s* - optional whitespace
    # 10 - literal "10"
    pattern = r'(\d+)\s*/\s*10'
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    return None

In [13]:
def compute_avg_dog_rating(num_posts):
    # TODO: complete
    total_score = 0
    num_scores = 0
    for i, feed_view in enumerate(prof_feed.feed[:num_posts]):
        score = extract_score(feed_view.post.record.text)
        if score is not None:
            # print(f"Found score: {score}/10")
            total_score += score
            num_scores += 1
        # else:
            # print("No score found in the format X/10")
    return total_score / num_scores if num_scores > 0 else 0

print("The average rating is:", compute_avg_dog_rating(100))

The average rating is: 13.0


## Exercise: Dog names
Collect the names of dogs within the latest 100 posts and print them to the
console. Hint: see if you can identify a pattern in the posts.

In [15]:
def extract_names(text):
    # This pattern looks for:
    # (?<![\.\?\!]\s) - negative lookbehind for period/question mark/exclamation followed by whitespace
    # (?<!\A) - negative lookbehind for start of string
    # \b[A-Z][a-zA-Z]*\b - word boundary, capital letter, followed by any letters, word boundary
    pattern = r'(?<![\.\?\!]\s)(?<!\A)\b[A-Z][a-zA-Z]*\b'
    
    matches = re.finditer(pattern, text)
    return [match.group() for match in matches]

In [16]:
def collect_dog_names(num_posts):
    # TODO: complete
    all_names = []
    for i, feed_view in enumerate(prof_feed.feed[:num_posts]):
        names = extract_names(feed_view.post.record.text)
        if names:
            all_names.extend(names)
    return all_names

print("Here are the dog_names:", collect_dog_names(100))

Here are the dog_names: ['George', 'SeniorPupSaturday', 'Fiyero', 'Dasia', 'The', 'Farmers', 'Dog', 'Top', 'Dogs', 'Dill', 'GIANT', 'Roger', 'Roger', 'ER', 'Panda', 'We', 'Panda', 'Panda', 'I', 'Ellie', 'Flipper', 'ALL', 'Good', 'Lucie', 'Muamba', 'Ollie', 'Heckles', 'ONLY', 'April', 'Top', 'WORST', 'Dogs', 'Joey', 'Bambi', 'Top', 'Dogs', 'March', 'Pippa', 'SeniorPupSaturday', 'Thank', 'Butterfly', 'Butterfly', 'Top', 'Dogs', 'Leon', 'Hubie', 'Fig', 'Oreo', 'FBI', 'Choco', 'We', 'ADOPTED', 'Choco', 'Rex']


## Exercise: Soliciting donations
Some posts from the WeRateDogs account ask for donations -- usually for
covering medical costs for the featured dogs. Within the latest 100 posts, print
the text content of those that fall into this category

In [ ]:
def donation_posts(num_posts):
    # TODO: complete
    return []

for post_text in donation_posts(100):
    print(post_text)

In [53]:
"""Implementation of automated moderator"""

from typing import List
from atproto import Client
from dotenv import load_dotenv
import os
import pprint
import pandas as pd


load_dotenv(override=True)
USERNAME = os.getenv("USERNAME")
PW = os.getenv("PW")

client = Client()
profile = client.login(USERNAME, PW)
pprint.pprint(profile.__dict__)

T_AND_S_LABEL = "t-and-s"
DOG_LABEL = "dog"
THRESH = 0.3

class AutomatedLabeler:
    """Automated labeler implementation"""

    def __init__(self, client: Client, input_dir):
        self.client = client

        # Read domains and words files with headers
        domains_df = pd.read_csv('./bluesky-assign3/labeler-inputs/t-and-s-domains.csv')
        self.domains = {domain.lower(): True for domain in domains_df['Domain'].dropna()}
        words_df = pd.read_csv('./bluesky-assign3/labeler-inputs/t-and-s-words.csv')
        self.words = {word.lower(): True for word in words_df['Word'].dropna()}
   
    def moderate_post(self, url: str) -> List[str]:
        """
        Apply moderation to the post specified by the given url
        """
        # Labeling logic
        # Milestone 2: Apply "t-and-s" label
        return []
    
    def post_from_url(client: Client, url: str):
        """
        Retrieve a Bluesky post from its URL
        """
        parts = url.split("/")
        rkey = parts[-1]
        handle = parts[-3]
        return client.get_post(rkey, handle)

    # post = post_from_url(client, "https://bsky.app/profile/labeler-test.bsky.social/post/3lktj7ewxxv2q")
    # pprint.pprint(post.value.__dict__)

    # Milestone 2: Label posts with T&S words and domains
    def find_matches(self, text: str) -> dict:
        """Find matches in text from both domains and words lists"""
        text = text.lower()
        domain_matches = [domain for domain in self.domains if domain in text]
        word_matches = [word for word in self.words if word.lower() in text]
        
        return {
            'domain_matches': domain_matches,
            'word_matches': word_matches
        }
    
    def detect_t_and_s(self, url: str) -> List[str]:
        post = post_from_url(client, url)
        post_text = post.value.text.lower()
        print(post_text)

        # Find matches
        matches = self.find_matches(post_text)
        print(matches)


{'associated': ProfileAssociated(chat=None, feedgens=0, labeler=True, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'),
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:yzpplgm5kftdgpf2wsnrbgdn/bafkreih3fpryxoepb44fzyr3sfn32fr7fqqka4kle6h4not7jlwtdvzghe@jpeg',
 'banner': None,
 'created_at': '2025-02-13T17:12:12.845Z',
 'description': None,
 'did': 'did:plc:yzpplgm5kftdgpf2wsnrbgdn',
 'display_name': '',
 'followers_count': 0,
 'follows_count': 1,
 'handle': 'trustylabeler.bsky.social',
 'indexed_at': '2025-02-13T17:12:12.845Z',
 'joined_via_starter_pack': None,
 'labels': [],
 'pinned_post': None,
 'posts_count': 0,
 'py_type': 'app.bsky.actor.defs#profileViewDetailed',
 'viewer': ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState')}


In [55]:
testlabel = AutomatedLabeler(Client, "./test-data/input-posts-t-and-s.csv")

testlabel.detect_t_and_s("https://bsky.app/profile/labeler-test.bsky.social/post/3lktfcjgyau2v")

testlabel.detect_t_and_s("https://bsky.app/profile/labeler-test.bsky.social/post/3lktg53dyqe2e")

community standards are the backbone of healthy online spaces. without clear guidelines, platforms struggle to balance freedom of expression with safety by design principles that protect their most vulnerable users.
{'domain_matches': [], 'word_matches': ['community standards', 'platform', 'safety by design']}
just finished planting my spring garden! anyone else getting their hands dirty this weekend? tomatoes, basil, and zucchini are in the ground and i'm already dreaming of summer salads.
{'domain_matches': [], 'word_matches': []}
